In [5]:
import nltk
import itertools
import numpy as np
import csv
import pickle
import os
# sentence_start_token = 'start'
# sentence_end_token = 'end'
# unknown_token = 'unknown'
# text = 'Oh! my god, I dont know why did I buy this product, A waste of momney. This is outrageous, I dont get it. \
# My name is Sam Helsen and I like trekking. What is your name? Are you Sam Helsen?. I am going to America, the land of free.\
#  Last night I saw a bad man. Sam Helsen was caught smoking weed amid the crowd, this is indeed an outrageous attempt. The police charged fine, such a poor fellow. \
#  Sam has lost a lot of money in gambling and now he is loosing by paying fines. Sam bought a product yesterday, which was infact \
#  an expire product. He applied it yesterday to his face and got allergic itching. Such a unlucky fellow. He ones agin lost a lot of money \
#  treating the ailment. '   
data_path = "/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/reddit-comments-2015-08.csv"

vocab_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token = "SENTENCE_END"
 
# Read the data and append SENTENCE_START and SENTENCE_END tokens
print ("Reading CSV file...")
with open(data_path, 'r') as f:
    reader = csv.reader(f, skipinitialspace=True)
    next(reader)
    # Split full comments into sentences
    sentences = itertools.chain(*[nltk.sent_tokenize(x[0].lower()) for x in reader])
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print ("Parsed %d sentences." % (len(sentences)))
     

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences]
 
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print ("Found %d unique words tokens." % len(word_freq.items()))
 
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocab_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print ("Using vocabulary size %d." % vocab_size)
print ("The least frequent word in our vocabulary is '%s' and appeared %d times." % (vocab[-1][0], vocab[-1][1]))
 
# Replace all words not in our vocabulary with the unknown token
for i, sent in enumerate(tokenized_sentences):
    tokenized_sentences[i] = [w if w in word_to_index else unknown_token for w in sent]
    
print ("\nExample sentence: '%s'" % sentences[0])
print ("\nExample sentence after Pre-processing: '%s'" % tokenized_sentences[0])
 
# Create the training data
X_train = np.asarray([[word_to_index[w] for w in sent[:-1]] for sent in tokenized_sentences])
y_train = np.asarray([[word_to_index[w] for w in sent[1:]] for sent in tokenized_sentences])


cleaned_dataset_path = '/Users/sam/All-Program/App-DataSet/Deep-Neural-Nets/Word-Search-NNets/reddit-comments-2015-08.p'

try:
    f = open(cleaned_dataset_path, 'wb')
    dataset_cleaned = {
        'X_train': X_train,
        'y_train': y_train
    }
    pickle.dump(dataset_cleaned, f, pickle.HIGHEST_PROTOCOL)
    f.close()
except Exception as e:
    print('Unable to save data to', cleaned_dataset_path, ':', e)
    raise

    
statinfo = os.stat(cleaned_dataset_path)
print('Compressed pickle size:', statinfo.st_size)


Reading CSV file...
Parsed 79170 sentences.
Found 65752 unique words tokens.
Using vocabulary size 8000.
The least frequent word in our vocabulary is 'pie' and appeared 10 times.

Example sentence: 'SENTENCE_START i joined a new league this year and they have different scoring rules than i'm used to. SENTENCE_END'

Example sentence after Pre-processing: '['SENTENCE_START', 'i', 'joined', 'a', 'new', 'league', 'this', 'year', 'and', 'they', 'have', 'different', 'scoring', 'rules', 'than', 'i', "'m", 'used', 'to', '.', 'SENTENCE_END']'
Compressed pickle size: 8233872


In [15]:
vocab_size = 8000
unknown_token = "UNKNOWN_TOKEN"
sentence_start_token = "START"
sentence_end_token = "END"

with open(cleaned_dataset_path, 'rb') as f:
    data = pickle.load(f)
    X_train = (data['X_train'])
    y_train = (data['y_train'])
    
# Assign instance variables
no_output_units = vocab_size
no_input_units = vocab_size
no_hidden_units = 100
# Randomly initialize the network parameters, 
# We initialize the wights as such to avoid vanishing weights and weights explosion
U = np.random.uniform(-np.sqrt(1./no_input_units), np.sqrt(1./no_input_units), (no_input_units, no_hidden_units))
W = np.random.uniform(-np.sqrt(1./no_hidden_units), np.sqrt(1./no_hidden_units), (no_hidden_units, no_hidden_units))
V = np.random.uniform(-np.sqrt(1./no_hidden_units), np.sqrt(1./no_hidden_units), (no_hidden_units, no_output_units))
    

print (-np.sqrt(1./word_dim))
print ('')
print ('The shape of U weight matrix is: ', U.shape)
print ('The shape of W weight matrix is: ', W.shape)
print ('The shape of V weight matrix is: ', V.shape)

-0.0111803398875

The shape of U weight matrix is:  (8000, 100)
The shape of W weight matrix is:  (100, 100)
The shape of V weight matrix is:  (100, 8000)


In [54]:

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


# The hidden state would be in the dimension of [no_of_sequence x no_hidden_units]
def fprop(x):
    # The total number of time steps
    no_of_sequence = len(x)
    # During forward propagation we save all hidden states in s because need them later.
    # We add one additional element for the initial hidden, which we set to 0
    hidden_layer_state = np.zeros((no_of_sequence + 1, no_hidden_units))
    # We just define s[-1] as a zero, vector. because we look back one sequence back while computig the inputs_to_hid_layer at time t=0
    hidden_layer_state[-1] = np.zeros(hidden_dim)
    output_layer_state = np.zeros((no_of_sequence, no_output_units))

    for t in np.arange(no_of_sequence):
        # Note that we are indxing U by x[t]. This is the same as multiplying U with a one-hot vector.
        inputs_to_hid_layer = U[x[t],:] + W.dot(hidden_layer_state[t-1]) 
        # Here U[x[t],:] would just output the correcsponding row from U. This is equivallend of taking the dot
        # product of the U matrix with the one hot vector for the corresponding word.
        hidden_layer_state[t] = np.tanh(inputs_to_hid_layer)
        #print (hidden_layer_state[t].shape)
        
        imputs_to_output_layer = (hidden_layer_state[t].T).dot(V)
        output_layer_state[t] =  softmax(imputs_to_output_layer)
        #print (output_layer_state[t].shape)
    return [hidden_layer_state, output_layer_state]


# Test for One training sample
x = X_train[10]
print ('The no of words in x is: ', len(x))
hidden_layer_state, output_layer_state = fprop(x[0:4])
print (hidden_layer_state.shape, output_layer_state.shape)

The no of words in x is:  45
(5, 100) (4, 8000)


In [56]:
# Calculating the LOSS usinf Cross-Entropy

# Remember X_train is an array of word indices and so is the y
def cal_total_loss_CE(x, y):  # Using cross-entropy to calculate the total loss.
    CE_tot = 0
    for datarow_arr, label_arr in zip(x, y):
        # datarow_arr is such a array of sequence of word indices of a sentence.
        # Example: datarow = [0,4,7] then label = [4,7,1], where 0 indicates "start" and 1 indicates 'end'
        hidden_layer_state, output_layer_state = fprop(datarow_arr)
        # The above code will find the output_layer_state for that 1 sentence (datarow_arr).
        prediction = output_layer_state[np.arange(len(label_arr)), label_arr]

        CE_tot += -1 * np.sum(np.log(prediction))
        # For a cross entropy we would want to get the dot product of the one_hot matrix formed by the actuls labels
        # and the log(output_layer_state) but that would result in creating a very large one-hot matrix of vocab_size 8000.
        # Finding the values in the columns of output_layer_state for indices in label_arr is the equivallent of doing the dot product
    return CE_tot
 
def calculate_avg_loss(x, y):
    # Divide the total loss by the number of training examples
    N = np.sum((len(y_i) for y_i in y))
    return cal_total_loss_CE(x,y)/N
 
    
# For 2 examples
print ("Actual loss for %d dataset:"%2,  calculate_avg_loss([X_train[10][0:3], X_train[11][0:3]], [y_train[10][0:3], y_train[11][0:3]]))

# For the first 1000 sentences or dataset
print ("Expected Loss for random predictions: %f" % np.log(vocabulary_size))
print ("Average loss for %d dataset" %1000, calculate_avg_loss(X_train[:1000], y_train[:1000]))


Actual loss for 2 dataset: 8.98386077269
Expected Loss for random predictions: 8.987197
Average loss for 1000 dataset 8.98729075289


In [10]:
bptt_truncate = 3

def bptt(x, y):
    T = len(y)
    # Perform forward propagation
    o, s = forward_propagation(x)
    print (o.shape, s.shape)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(U.shape)
    dLdV = np.zeros(V.shape)
    dLdW = np.zeros(W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    print (delta_o.shape)
    # For each output backwards...
    # Backpropogation works from right to left, so we reverse the arrat, ex [0,1,2,3] be becomes [3,2,1,0]
    for t in np.arange(T)[::-1]:
        print ('')
        print ('aaaaaaaaaaaaaaaaa ', t)
        dLdV += np.outer(delta_o[t], s[t].T)
#         print ('s[t].T.shape ', s[t].T.shape)
#         print ('dLdV.shape ',dLdV.shape)
#         print ('V.T.shape ', V.T.shape)
#         print ('delta_o[t].shape ', delta_o[t].shape)
#         print ('')
        
#         print ('at t= %d s(t) is : '%t, 1 - (s[t] ** 2))
#         print ('at t= %d delta_o[t] is: '%t, delta_o[t])
#         print (delta_o[t]) * (1 - (s[t] ** 2))
#         delta_t = V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
#         print (delta_t)
#         print ('t-bptt_truncate is: ', t-bptt_truncate)
#         print ('max(0, t-bptt_truncate) is: ',max(0, t-bptt_truncate))
#         print ('np.arange(max(0, t-self.bptt_truncate), t+1) ', np.arange(max(0, t-bptt_truncate), t+1))
#         print ('np.arange(max(0, t-self.bptt_truncate), t+1)[::-1] ', np.arange(max(0, t-bptt_truncate), t+1)[::-1])
#         #         print (np.arange(max(0, t-bptt_truncate), t+1)[::-1])
#         for bptt_step in np.arange(max(0, t-bptt_truncate), t+1)[::-1]:
#             print ('popopopopoo ', bptt_step)
#             print (s[bptt_step-1])

        # Initial delta calculation
        delta_t = V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-bptt_truncate), t+1)[::-1]:
            print ('bbbbbbbbbbbbbbbbbb ', bptt_step)
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])  
            print ('s[bptt_step-1]', s[bptt_step-1])
            print ('dLdW',  dLdW)
#             dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
            print ('delta_t ', delta_t)
        break
#     return [dLdU, dLdV, dLdW]

# We take a small example to 

bptt([0,1,2], [1,2,3]) # 0,1,2,3,4 is the word sequence for example:  0->I, 1->am, 2->going, 3->to, 4->hit, 5-> you 

Expected Loss for random predictions: 8.987197
Actual loss: 8.987314


In [11]:
bptt_truncate = 3

def bptt(x, y):
    T = len(y)
    # Perform forward propagation
    o, s = forward_propagation(x)
    print (o.shape, s.shape)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(U.shape)
    dLdV = np.zeros(V.shape)
    dLdW = np.zeros(W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    print (delta_o.shape)
    # For each output backwards...
    # Backpropogation works from right to left, so we reverse the arrat, ex [0,1,2,3] be becomes [3,2,1,0]
    for t in np.arange(T)[::-1]:
        print ('')
        print ('aaaaaaaaaaaaaaaaa ', t)
        dLdV += np.outer(delta_o[t], s[t].T)
#         print ('s[t].T.shape ', s[t].T.shape)
#         print ('dLdV.shape ',dLdV.shape)
#         print ('V.T.shape ', V.T.shape)
#         print ('delta_o[t].shape ', delta_o[t].shape)
#         print ('')
        
#         print ('at t= %d s(t) is : '%t, 1 - (s[t] ** 2))
#         print ('at t= %d delta_o[t] is: '%t, delta_o[t])
#         print (delta_o[t]) * (1 - (s[t] ** 2))
#         delta_t = V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
#         print (delta_t)
#         print ('t-bptt_truncate is: ', t-bptt_truncate)
#         print ('max(0, t-bptt_truncate) is: ',max(0, t-bptt_truncate))
#         print ('np.arange(max(0, t-self.bptt_truncate), t+1) ', np.arange(max(0, t-bptt_truncate), t+1))
#         print ('np.arange(max(0, t-self.bptt_truncate), t+1)[::-1] ', np.arange(max(0, t-bptt_truncate), t+1)[::-1])
#         #         print (np.arange(max(0, t-bptt_truncate), t+1)[::-1])
#         for bptt_step in np.arange(max(0, t-bptt_truncate), t+1)[::-1]:
#             print ('popopopopoo ', bptt_step)
#             print (s[bptt_step-1])

        # Initial delta calculation
        delta_t = V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-bptt_truncate), t+1)[::-1]:
            print ('bbbbbbbbbbbbbbbbbb ', bptt_step)
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            dLdW += np.outer(delta_t, s[bptt_step-1])  
            print ('s[bptt_step-1]', s[bptt_step-1])
            print ('dLdW',  dLdW)
#             dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step
            delta_t = W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
            print ('delta_t ', delta_t)
        break
#     return [dLdU, dLdV, dLdW]

# We take a small example to 

bptt([0,1,2], [1,2,3]) # 0,1,2,3,4 is the word sequence for example:  0->I, 1->am, 2->going, 3->to, 4->hit, 5-> you 

(3, 8000) (4, 100)
(3, 8000)

aaaaaaaaaaaaaaaaa  2
bbbbbbbbbbbbbbbbbb  2
s[bptt_step-1] [-0.00282782 -0.00907338 -0.01465256  0.00252337 -0.00382819  0.01635668
  0.01294681 -0.0087975   0.00392096 -0.00873958  0.00309464 -0.00527214
 -0.01122418  0.00481832 -0.00731619 -0.00896967 -0.01245067  0.00612713
  0.00305838  0.00427345 -0.0033523  -0.00328978  0.01520627 -0.00840482
  0.00568465  0.01520935  0.00343978 -0.00189767 -0.0042385   0.00297322
 -0.00317739 -0.01137146  0.00101308 -0.00427126  0.01046035 -0.00404881
  0.00497765 -0.00338904  0.00126298  0.00519548  0.00594546 -0.01020443
  0.00113324 -0.00168416 -0.00573876  0.00343812  0.0019591   0.0009989
  0.00307361 -0.00879284  0.0086315   0.01064532  0.00254845  0.00365477
  0.00803679  0.00463294 -0.01071622 -0.00732892 -0.00702249 -0.00749406
  0.00489553 -0.00900501 -0.00458312  0.0084647  -0.0129267  -0.0016163
 -0.01029885  0.01285934  0.00813531 -0.00061622 -0.01112088 -0.00611581
 -0.00277355 -0.00056213 -0.00795453  

In [87]:
a = np.array([1,2,3,4])
b = np.array([10,11,12])
print (a.shape, b.shape)
print (np.outer(a,b))

(4,) (3,)
[[10 11 12]
 [20 22 24]
 [30 33 36]
 [40 44 48]]


In [ ]:
def gradient_check(self, x, y, h=0.001, error_threshold=0.01):
    # Calculate the gradients using backpropagation. We want to checker if these are correct.
    bptt_gradients = self.bptt(x, y)
    # List of all parameters we want to check.
    model_parameters = ['U', 'V', 'W']
    # Gradient check for each parameter
    for pidx, pname in enumerate(model_parameters):
        # Get the actual parameter value from the mode, e.g. model.W
        parameter = operator.attrgetter(pname)(self)
        print "Performing gradient check for parameter %s with size %d." % (pname, np.prod(parameter.shape))
        # Iterate over each element of the parameter matrix, e.g. (0,0), (0,1), ...
        it = np.nditer(parameter, flags=['multi_index'], op_flags=['readwrite'])
        while not it.finished:
            ix = it.multi_index
            # Save the original value so we can reset it later
            original_value = parameter[ix]
            # Estimate the gradient using (f(x+h) - f(x-h))/(2*h)
            parameter[ix] = original_value + h
            gradplus = self.calculate_total_loss([x],[y])
            parameter[ix] = original_value - h
            gradminus = self.calculate_total_loss([x],[y])
            estimated_gradient = (gradplus - gradminus)/(2*h)
            # Reset parameter to original value
            parameter[ix] = original_value
            # The gradient for this parameter calculated using backpropagation
            backprop_gradient = bptt_gradients[pidx][ix]
            # calculate The relative error: (|x - y|/(|x| + |y|))
            relative_error = np.abs(backprop_gradient - estimated_gradient)/(np.abs(backprop_gradient) + np.abs(estimated_gradient))
            # If the error is to large fail the gradient check
            if relative_error &gt; error_threshold:
                print "Gradient Check ERROR: parameter=%s ix=%s" % (pname, ix)
                print "+h Loss: %f" % gradplus
                print "-h Loss: %f" % gradminus
                print "Estimated_gradient: %f" % estimated_gradient
                print "Backpropagation gradient: %f" % backprop_gradient
                print "Relative Error: %f" % relative_error
                return
            it.iternext()
        print "Gradient check for parameter %s passed." % (pname)
 
gradient_check = gradient_check
 
# To avoid performing millions of expensive calculations we use a smaller vocabulary size for checking.
grad_check_vocab_size = 100
np.random.seed(10)
model = RNNNumpy(grad_check_vocab_size, 10, bptt_truncate=1000)
model.gradient_check([0,1,2,3], [1,2,3,4])